# Designing and Creating a Database

The goal of this project is to meticulously design and create a database, through a process that will involve:

1. Importing Data into SQL
2. Designing a normalized Database Schema
3. Creating Tables for our Schema
4. Inserting Data into our Schema

We will work with data from Major League Baseball games compiled by [Retrosheet](https://www.retrosheet.org/). The main file we will work from is `game_log.csv`, which has been compiled and pre-cleaned from 127 separate CSV files by Retrosheet. This file has hundreds of data points on each game. 

What we want to do is to convert and normalize this data into several separate tables using SQL, and create a robust database of game-level statistics. 

In addition to the main `game_log.csv` file, we also have the following 3 helper files:

- `park_codes.csv`
- `person_codes.csv`
- `team_codes.csv`

These 3 helper files will form the basis for 3 of our normalized tables.

We'll start by importing some of the libraries we'll need, and then reading in the csv files.

## Reading in the files

In [1]:
library(tidyverse)
library(RSQLite)
library(DBI)

Registered S3 method overwritten by 'rvest':
  method            from
  read_xml.response xml2
-- Attaching packages --------------------------------------- tidyverse 1.2.1 --
v ggplot2 3.2.0     v purrr   0.3.2
v tibble  2.1.3     v dplyr   0.8.3
v tidyr   0.8.3     v stringr 1.4.0
v readr   1.3.1     v forcats 0.4.0
Warning message:
"package 'dplyr' was built under R version 3.6.1"-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()
Warning message:
"package 'RSQLite' was built under R version 3.6.1"

In [2]:
# Since R has some trouble guessing certain column names, we'll explicitly tell 
# it the column type in the columns where there is a problem. 

log <- read_csv("game_log.csv", 
                col_types = cols(.default = "c",
                 v_league = "c", h_league = "c",
                 `3b_umpire_id` = "c", `3b_umpire_name` = "c",
                 `2b_umpire_id` = "c", `2b_umpire_name` = "c",
                 `lf_umpire_id` = "c", `lf_umpire_name` = "c",
                 `rf_umpire_id` = "c", `rf_umpire_name` = "c",
                 completion = "c", winning_rbi_batter_id = "c",
                 winning_rbi_batter_id_name = "c", protest = "c",
                 v_first_catcher_interference = "c", 
                 h_first_catcher_interference = "c"))

head(log)

print(dim(log))

date,number_of_game,day_of_week,v_name,v_league,v_game_number,h_name,h_league,h_game_number,v_score,...,h_player_7_name,h_player_7_def_pos,h_player_8_id,h_player_8_name,h_player_8_def_pos,h_player_9_id,h_player_9_name,h_player_9_def_pos,additional_info,acquisition_info
18710504,0,Thu,CL1,NA,1,FW1,NA,1,0,...,Ed Mincher,7,mcdej101,James McDermott,8,kellb105,Bill Kelly,9,NA,Y
18710505,0,Fri,BS1,NA,1,WS3,NA,1,20,...,Asa Brainard,1,burrh101,Henry Burroughs,9,berth101,Henry Berthrong,8,HTBF,Y
18710506,0,Sat,CL1,NA,2,RC1,NA,1,12,...,Pony Sager,6,birdg101,George Bird,7,stirg101,Gat Stires,9,NA,Y
18710508,0,Mon,CL1,NA,3,CH1,NA,1,12,...,Ed Duffy,6,pinke101,Ed Pinkham,5,zettg101,George Zettlein,1,NA,Y
18710509,0,Tue,BS1,NA,2,TRO,NA,1,9,...,Steve Bellan,5,pikel101,Lip Pike,3,cravb101,Bill Craver,6,HTBF,Y
18710511,0,Thu,CH1,NA,2,CL1,NA,4,18,...,Ezra Sutton,5,carlj102,Jim Carleton,3,bassj101,John Bass,6,NA,Y


[1] 171907    161


The game log has a record of over 170,000 games, represented by the rows. It looks like these games are chronologically ordered by the very first column that denotes date, `date`, in a "YYYYMMDD" format. The games occur between 1871 and 2016.

For each game we have:

* general information on the game
* team level stats for each team
* a list of players from each team, numbered, with their defensive positions
* the umpires that officiated the game
* some 'awards', like winning and losing pitcher

We have a `game_log_fields.txt` file that tells us that the player number corresponds with the order in which they batted.

It's also worth noting that there is no natural primary key column for this table.

In [3]:
person <- read_csv("person_codes.csv")

head(person)

dim(person)

Parsed with column specification:
cols(
  id = col_character(),
  last = col_character(),
  first = col_character(),
  player_debut = col_character(),
  mgr_debut = col_character(),
  coach_debut = col_character(),
  ump_debut = col_character()
)


id,last,first,player_debut,mgr_debut,coach_debut,ump_debut
aardd001,Aardsma,David,04/06/2004,NA,NA,NA
aaroh101,Aaron,Hank,04/13/1954,NA,NA,NA
aarot101,Aaron,Tommie,04/10/1962,NA,04/06/1979,NA
aased001,Aase,Don,07/26/1977,NA,NA,NA
abada001,Abad,Andy,09/10/2001,NA,NA,NA
abadf001,Abad,Fernando,07/28/2010,NA,NA,NA


[1] 20494     7

The `person` table looks to be a list of people with IDs. 

The IDs look like they match up with those used in the game log. There are debut dates, for players, managers, coaches and umpires. We can see that some people might have been one or more of these roles.

It also looks like coaches and managers are two different things in baseball. After some research, managers are what would be called a 'coach' or 'head coach' in other sports, and coaches are more specialized, like base coaches. 

It also seems like coaches aren't recorded in the main `game_log.csv` file.

In [4]:
park <- read_csv("park_codes.csv")

head(park)

dim(park)

Parsed with column specification:
cols(
  park_id = col_character(),
  name = col_character(),
  aka = col_character(),
  city = col_character(),
  state = col_character(),
  start = col_character(),
  end = col_character(),
  league = col_character(),
  notes = col_character()
)


park_id,name,aka,city,state,start,end,league,notes
ALB01,Riverside Park,NA,Albany,NY,09/11/1880,05/30/1882,NL,TRN:9/11/80;6/15&9/10/1881;5/16-5/18&5/30/1882
ALT01,Columbia Park,NA,Altoona,PA,04/30/1884,05/31/1884,UA,NA
ANA01,Angel Stadium of Anaheim,Edison Field; Anaheim Stadium,Anaheim,CA,04/19/1966,NA,AL,NA
ARL01,Arlington Stadium,NA,Arlington,TX,04/21/1972,10/03/1993,AL,NA
ARL02,Rangers Ballpark in Arlington,The Ballpark in Arlington; Ameriquest Fl,Arlington,TX,04/11/1994,NA,AL,NA
ATL01,Atlanta-Fulton County Stadium,NA,Atlanta,GA,04/12/1966,09/23/1996,NL,NA


[1] 252   9

This `park` table is a list of all baseball parks, and includes main names (as well as other name variants). There are IDs which seem to match with the game log, as well as names, nicknames, city and league.

In [5]:
team <- read_csv("team_codes.csv")

head(team)

dim(team)

Parsed with column specification:
cols(
  team_id = col_character(),
  league = col_character(),
  start = col_double(),
  end = col_double(),
  city = col_character(),
  nickname = col_character(),
  franch_id = col_character(),
  seq = col_double()
)


team_id,league,start,end,city,nickname,franch_id,seq
ALT,UA,1884,1884,Altoona,Mountain Cities,ALT,1
ARI,NL,1998,0,Arizona,Diamondbacks,ARI,1
BFN,NL,1879,1885,Buffalo,Bisons,BFN,1
BFP,PL,1890,1890,Buffalo,Bisons,BFP,1
BL1,NA,1872,1874,Baltimore,Canaries,BL1,1
BL2,AA,1882,1891,Baltimore,Orioles,BL2,1


[1] 150   8

The `team` table seems to be a list of all teams, with team_ids which seem to match the game log.

### Defensive Positions

In the game log, each player has a defensive position listed, which seems to be a number between 1-10. This [article](http://probaseballinsider.com/baseball-instruction/baseball-basics/baseball-basics-positions/) gives us a list of names for each numbered position:

* Pitcher
* Catcher
* 1st Base
* 2nd Base
* 3rd Base
* Shortstop
* Left Field
* Center Field
* Right Field

The 10th position isn't included - it may be a way of describing a designated hitter that does not field. We will make this an 'Unknown Position'.

### Leagues

Wikipedia tells us there are currently two leagues - the American (AL) and National (NL). 

Upon further research, we see that there are actually 4 more in addition to those two:

* NL: National League
* AL: American League
* AA: [American Association](https://en.wikipedia.org/wiki/American_Association_%2819th_century%29)
* FL: [Federal League](https://en.wikipedia.org/wiki/Federal_League)
* PL: [Players League](https://en.wikipedia.org/wiki/Players%27_League)
* UA: [Union Association](https://en.wikipedia.org/wiki/Union_Association)

It also looks like we have about 1000 games where the home team doesn't have a value for league.

## Importing Data into SQLite 

We'll use `dbWriteTable()` to create tables for each of our CSV files into a new SQLite database, `mlb.db`.

The table name for each file will be the same as the CSV file name.

We'll also create a new column in the `game_log` table called `game_id`. This new `game_id` column will be made via string concatenation as highlighted [here](https://www.retrosheet.org/eventfile.htm) in the retrosheet data dictionary:

"***id***: *Each game begins with a twelve character ID record which identifies the date, home team, and number of the game. For example, `ATL198304080` should be read as follows. The first three characters identify the home team (the Braves). The next four are the year (1983). The next two are the month (April) using the standard numeric notation, 04, followed by the day (08). The last digit indicates if this is a single game (0), first game (1) or second game (2) if more than one game is played during a day, usually a double header The id record starts the description of a game thus ending the description of the preceding game in the file*"

This new `game_id` column will serve as a unique column in the `game_log` table.

In [6]:
conn <- dbConnect(SQLite(), "mlb.db")

dbWriteTable(conn = conn, name = "game_log", value = log, row.names = FALSE, header = TRUE)

dbWriteTable(conn = conn, name = "person_codes", value = person, row.names = FALSE, header = TRUE)

dbWriteTable(conn = conn, name = "team_codes", value = team, row.names = FALSE, header = TRUE)

dbWriteTable(conn = conn, name = "park_codes", value = park, row.names = FALSE, header = TRUE)

Let's confirm that all the tables are in, and with the correct names.

In [7]:
dbListTables(conn)

[1] "appearance_type"   "game"              "game_log"         
 [4] "league"            "park"              "park_codes"       
 [7] "person"            "person_appearance" "person_codes"     
[10] "team"              "team_appearance"   "team_codes"

In [8]:
game_id_col_add <-"
ALTER TABLE game_log
ADD COLUMN game_id TEXT;"

dbExecute(conn, game_id_col_add)

[1] 0

After having created the `game_id` column above, we'll now populate it with values using string concatenation:

In [9]:
game_log_fill <- "
UPDATE game_log
SET game_id = date || h_name || number_of_game
WHERE game_id IS NULL;"

dbExecute(conn, game_log_fill)

[1] 171907

In [10]:
dbGetQuery(conn, "SELECT game_id, date, h_name, number_of_game FROM game_log LIMIT 10")

game_id,date,h_name,number_of_game
18710504FW10,18710504,FW1,0
18710505WS30,18710505,WS3,0
18710506RC10,18710506,RC1,0
18710508CH10,18710508,CH1,0
18710509TRO0,18710509,TRO,0
18710511CL10,18710511,CL1,0
18710513CL10,18710513,CL1,0
18710513FW10,18710513,FW1,0
18710515FW10,18710515,FW1,0
18710516BS10,18710516,BS1,0


Looking at a preview of the modifier game_log table above, we confirm that we constructed a `game_id` column in the format: "date" + "home team name" + "number of game", where date is given in YYYYMMDD format, home team name is a 3-character abbreviation, and number of game indicates the number of games played.

##  Normalization of Data

Normalization helps us reduce redundancy and improve data integrity in our database.

Some specific opportunities for normalization we should focus on are the following:

* In `person_codes`, all the debut dates will be able to be reproduced using `game_log` data.
* In `team_codes`, the start, end and sequence columns will be able to be reproduced using `game_log` data.
* In `park_codes`, the start and end years will be able to be reproduced using `game_log` data. While technically the state is an attribute of the city, we might not want to have a an incomplete city/state table so we will leave this in.
* There are lots of places in `game_log` where we have a player ID followed by the players name. We will be able to remove this and use the name data in person_codes
* In `game_log`, all offensive and defensive stats are repeated for the home team and the visiting team. We could break these out and have a table that lists each game twice, one for each team, and cut out this column repetition.
* Similarly, in `game_log`, we have a listing for 9 players on each team with their positions - we can remove these and have one table that tracks player appearances and their positions.
* We can do a similar thing with the umpires from `game_log`, instead of listing all four positions as columns, we can put the umpires either in their own table or make one table for players, umpires and managers.
* We have several awards in `game_log` like winning pitcher and losing pitcher. We can either break these out into their own table, have a table for awards, or combine the awards in with general appearances like the players and umpires.

### Schema designing

With the above points in mind, the following schema was devised, using the free schema builder on [dbdesigner.net](https://www.dbdesigner.net/) to construct the schema. The tables will be divided as follows:

![Schema](https://i.gyazo.com/869320f0dbe2516b3d465827733ad724.png)


## Database Building

### Creating tables without Foreign Key Relations

We will start by creating the tables without foreign key relations. The tables we will thus create first are the following:

- ***person***

The 'debut' columns will be ommitted, since that information is repeated on other tables. Data on coaches is not included since the `game_log` file has no info on that data.

- ***park***

The columns found in the `game_log` file, primarily the 'start', 'end', and 'league' columns, will be removed.

- ***league***

Older leagues might not be as well-known as the current two leagues. For this reason we'll designate a table to storing all league names.

- ***appearance_type***

The appearance table will include data on players with positions, umpires, managers, and awards (like winning pitcher). It also will store information on what kind of appearances are available. 

The `person`, `park`, and `appearance_type` tables will be created by making an empty table, and then populating it with values by using queries, selecting current values from the `person_codes`, `park_codes`, and `appearance_type.csv` table. In the `league` table, the values will have to be manually specified. 

The `person` table will be created first:

In [11]:
create_person <- "
CREATE TABLE IF NOT EXISTS person (
  person_id TEXT PRIMARY KEY,
  first_name TEXT,
  last_name TEXT
  );"
  
dbExecute(conn, create_person)

populate_person <- "
INSERT OR IGNORE INTO person
SELECT
  id,
  first,
  last
FROM person_codes;"
  
dbExecute(conn, populate_person)

[1] 0

[1] 0

Let's peview the `person` table we just made:

In [12]:
person_check <- "
SELECT * FROM person;"

person_preview <- dbGetQuery(conn, person_check)

head(person_preview, 10)

dim(person_preview)

person_id,first_name,last_name
aardd001,David,Aardsma
aaroh101,Hank,Aaron
aarot101,Tommie,Aaron
aased001,Don,Aase
abada001,Andy,Abad
abadf001,Fernando,Abad
abadj101,John,Abadie
abbae101,Ed,Abbaticchio
abbeb101,Bert,Abbey
abbec101,Charlie,Abbey


[1] 20494     3

The `person` table looks right. Now let's create the `park` table:

In [13]:
park_create <- "
CREATE TABLE IF NOT EXISTS park (
  park_id TEXT PRIMARY KEY,
  name TEXT,
  nickname TEXT,
  city TEXT,
  state TEXT,
  notes TEXT
);"

dbExecute(conn, park_create)

park_populate <- "
INSERT OR IGNORE INTO park
SELECT
  park_id,
  name,
  aka,
  city,
  state,
  notes
FROM park_codes;
"

dbExecute(conn, park_populate)

[1] 0

[1] 0

Let's also preview the `park` table:

In [14]:
park_check <- "SELECT * FROM park"

park_preview <- dbGetQuery(conn, park_check)

head(park_preview, 10)

park_id,name,nickname,city,state,notes
ALB01,Riverside Park,NA,Albany,NY,TRN:9/11/80;6/15&9/10/1881;5/16-5/18&5/30/1882
ALT01,Columbia Park,NA,Altoona,PA,NA
ANA01,Angel Stadium of Anaheim,Edison Field; Anaheim Stadium,Anaheim,CA,NA
ARL01,Arlington Stadium,NA,Arlington,TX,NA
ARL02,Rangers Ballpark in Arlington,The Ballpark in Arlington; Ameriquest Fl,Arlington,TX,NA
ATL01,Atlanta-Fulton County Stadium,NA,Atlanta,GA,NA
ATL02,Turner Field,NA,Atlanta,GA,NA
ATL03,Suntrust Park,NA,Atlanta,GA,NA
BAL01,Madison Avenue Grounds,NA,Baltimore,MD,WS3
BAL02,Newington Park,NA,Baltimore,MD,BL1:1872-74; BL4:1873; BL2: 1882


Next, we build the league table. This is where its values will have to be manually entered in:

In [15]:
league_create <- "
CREATE TABLE IF NOT EXISTS league (
  league_id TEXT PRIMARY KEY,
  name TEXT
);"

dbExecute(conn, league_create)

league_populate <- '
INSERT OR IGNORE INTO league
VALUES
  ("NL", "National League"),
  ("AL", "American League"),
  ("AA", "American Association"),
  ("FL", "Federal League"),
  ("PL", "Players League"),
  ("UA", "Union Association")
;'

dbExecute(conn, league_populate)

[1] 0

[1] 0

In [16]:
league_check <- "SELECT * FROM league"

league_preview <- dbGetQuery(conn, league_check)

league_preview

league_id,name
NL,National League
AL,American League
AA,American Association
FL,Federal League
PL,Players League
UA,Union Association


Now, the `appearance_type` table will have to be read in.

The CSV values will simply be manually entered in:

In [17]:
appearance_type <- read_csv('appearance_type_id,name,category
O1,Batter 1,offense
O2,Batter 2,offense
O3,Batter 3,offense
O4,Batter 4,offense
O5,Batter 5,offense
O6,Batter 6,offense
O7,Batter 7,offense
O8,Batter 8,offense
O9,Batter 9,offense
D1,Pitcher,defense
D2,Catcher,defense
D3,1st Base,defense
D4,2nd Base,defense
D5,3rd Base,defense
D6,Shortstop,defense
D7,Left Field,defense
D8,Center Field,defense
D9,Right Field,defense
D10,Unknown Position,defense
UHP,Home Plate,umpire
U1B,First Base,umpire
U2B,Second Base,umpire
U3B,Third Base,umpire
ULF,Left Field,umpire
URF,Right Field,umpire
MM,Manager,manager
AWP,Winning Pitcher,award
ALP,Losing Pitcher,award
ASP,Saving Pitcher,award
AWB,Winning RBI Batter,award
PSP,Starting Pitcher,pitcher')

dbWriteTable(conn = conn, 
             name = "appearance_type", 
             value = appearance_type,
             row.names = FALSE, 
             header = TRUE)

ERROR: Error: Table appearance_type exists in database, and both overwrite and append are FALSE


In [ ]:
head(dbGetQuery(conn, "SELECT * from appearance_type"),10)

### Creating the `game` and `team` tables

Because these 2 tables need to exist before `person_appearance` and `team_appearance`, the `game` and `team` tables will now be created, with the tables they reference kept in mind. 

Notice that the `day` column in `game` is being converted from a "D" or "N" value to a binary Boolean value of "0" or "1".

In [ ]:
game_create <- "
CREATE TABLE IF NOT EXISTS game (
  game_id TEXT PRIMARY KEY,
  date TEXT,
  number_of_game INTEGER,
  park_id TEXT,
  length_outs INTEGER,
  day BOOLEAN,
  completion TEXT,
  forefeit TEXT,
  protest TEXT,
  attendance INTEGER,
  legnth_minutes INTEGER,
  additional_info TEXT,
  acquisition_info TEXT,
  FOREIGN KEY (park_id) REFERENCES park(park_id)
);"

dbExecute(conn, game_create)

game_populate <- '
INSERT OR IGNORE INTO game
SELECT
  game_id,
  date,
  number_of_game,
  park_id,
  length_outs,
  CASE
    WHEN day_night = "D" THEN 1
    WHEN day_night = "N" THEN 0
    ELSE NULL
    END
    AS day,
  completion,
  forefeit,
  protest,
  attendance,
  length_minutes,
  additional_info,
  acquisition_info
FROM game_log;'

dbExecute(conn, game_populate)

In [ ]:
game_check <- "SELECT * FROM game LIMIT 10;"

game_preview <- dbGetQuery(conn, game_check)

game_preview

The `game` table turned out well. Let's also create `team`.

In [ ]:
team_create <- "
CREATE TABLE IF NOT EXISTS team (
  team_id TEXT PRIMARY KEY,
  league_id TEXT,
  city TEXT,
  nickname TEXT,
  franch_id TEXT,
  FOREIGN KEY (league_id) REFERENCES league(league_id)
);"
  
dbExecute(conn, team_create)

team_populate <- "
INSERT OR IGNORE INTO team
SELECT
  team_id,
  league,
  city,
  nickname,
  franch_id
FROM team_codes;"

dbExecute(conn, team_populate)

In [ ]:
team_check <- "SELECT * FROM team LIMIT 10;"

team_preview <- dbGetQuery(conn, team_check)

team_preview

With the `game` and `team` tables complete, we'll next focus on creating the `team_appearance` table.

### Creating `team_appearance` table

Again, for reference, below is the part of the Schema that outlines the `team_appearance` table and the other tables it references:

![team_appearance](https://i.gyazo.com/b11e82417def87682caf567eaa61a6d1.png)

It should be noted that there is a composite key made up of two primary keys.

In [ ]:
team_appearance_create <- "
CREATE TABLE if NOT EXISTS team_appearance (
  team_id TEXT,
  game_id TEXT,
  home BOOLEAN,
  league_id TEXT,
  score INTEGER,
  line_score TEXT,
  at_bats INTEGER,
  hits INTEGER,
  doubles INTEGER,
  triples INTEGER,
  homeruns INTEGER,
  rbi INTEGER,
  sacrifice_hits INTEGER,
  sacrifice_flies INTEGER,
  hit_by_pitch INTEGER,
  walks INTEGER,
  intentional_walks INTEGER,
  strikeouts INTEGER,
  stolen_bases INTEGER,
  caught_stealing INTEGER,
  grounded_into_double INTEGER,
  first_catcher_interference INTEGER,
  left_on_base INTEGER,
  pitchers_used INTEGER,
  individual_earned_runs INTEGER,
  team_earned_runs INTEGER,
  wild_pitches INTEGER,
  balks INTEGER,
  putouts INTEGER,
  assists INTEGER,
  errors INTEGER,
  passed_balls INTEGER,
  double_plays INTEGER,
  triple_plays INTEGER,
  PRIMARY KEY (team_id, game_id),
  FOREIGN KEY (team_id) REFERENCES team(team_id),
  FOREIGN KEY (game_id) REFERENCES game(game_id),
  FOREIGN KEY (team_id) REFERENCES team(team_id)
);"
  
dbExecute(conn, team_appearance_create)

Populating the `team_appearance` table will be done using a union clause to distinguish between whether the team was the home team or otherwise, a boolean value denote by '1' or '0' in the `home` column of `team_appearance`.

In [ ]:
team_appearance_populate <- "
INSERT OR IGNORE INTO team_appearance
  SELECT
      h_name,
      game_id,
      1 AS home,
      h_league,
      h_score,
      h_line_score,
      h_at_bats,
      h_hits,
      h_doubles,
      h_triples,
      h_homeruns,
      h_rbi,
      h_sacrifice_hits,
      h_sacrifice_flies,
      h_hit_by_pitch,
      h_walks,
      h_intentional_walks,
      h_strikeouts,
      h_stolen_bases,
      h_caught_stealing,
      h_grounded_into_double,
      h_first_catcher_interference,
      h_left_on_base,
      h_pitchers_used,
      h_individual_earned_runs,
      h_team_earned_runs,
      h_wild_pitches,
      h_balks,
      h_putouts,
      h_assists,
      h_errors,
      h_passed_balls,
      h_double_plays,
      h_triple_plays
  FROM game_log

UNION

  SELECT    
      v_name,
      game_id,
      0 AS home,
      v_league,
      v_score,
      v_line_score,
      v_at_bats,
      v_hits,
      v_doubles,
      v_triples,
      v_homeruns,
      v_rbi,
      v_sacrifice_hits,
      v_sacrifice_flies,
      v_hit_by_pitch,
      v_walks,
      v_intentional_walks,
      v_strikeouts,
      v_stolen_bases,
      v_caught_stealing,
      v_grounded_into_double,
      v_first_catcher_interference,
      v_left_on_base,
      v_pitchers_used,
      v_individual_earned_runs,
      v_team_earned_runs,
      v_wild_pitches,
      v_balks,
      v_putouts,
      v_assists,
      v_errors,
      v_passed_balls,
      v_double_plays,
      v_triple_plays
  from game_log;"
  

dbExecute(conn, team_appearance_populate)

We'll now preview the `team_appearance` table we just created:

In [ ]:
team_appearance_check <- "SELECT * FROM team_appearance LIMIT 10"

team_appearance_preview <- dbGetQuery(conn, team_appearance_check)

team_appearance_preview

### Creating `person_appearance` table

With the `team_appearance` table done, the final table we have to create now is the `person_appearance` table. The `person_appearance` table will be used to store information on appearances in games by managers, players, and umpires, as detailed in the `appearance_type` table.

In [ ]:
person_appearance_create <- "
CREATE TABLE person_appearance (
  appearance_id INTEGER PRIMARY KEY,
  person_id TEXT,
  team_id TEXT,
  game_id TEXT,
  appearance_type_id,
  FOREIGN KEY (person_id) REFERENCES person(person_id),
  FOREIGN KEY (team_id) REFERENCES team(team_id),
  FOREIGN KEY (game_id) REFERENCES game(game_id),
  FOREIGN KEY (appearance_type_id) REFERENCES appearance_type(appearance_type_id)
);"
  
dbExecute(conn, person_appearance_create)

A similar technique that we used to insert data into the `team_appearance` table. The key difference here though is that we will need to write much larger queries - one for each column, where before we were writing only one for each team. 

We will need to determine for each column what the `appearance_type_id` will be by cross-referencing the columns with the `appearance_type` table.

In [ ]:
person_appearance_populate <- '
INSERT OR IGNORE INTO person_appearance  (
  game_id,
  team_id,
  person_id,
  appearance_type_id
) 
SELECT
  game_id,
  NULL,
  hp_umpire_id,
  "UHP"
FROM game_log
WHERE hp_umpire_id IS NOT NULL    

UNION

SELECT
  game_id,
  NULL,
  [1b_umpire_id],
  "U1B"
FROM game_log
WHERE "1b_umpire_id" IS NOT NULL

UNION

SELECT
  game_id,
  NULL,
  [2b_umpire_id],
  "U2B"
FROM game_log
WHERE [2b_umpire_id] IS NOT NULL

UNION

SELECT
  game_id,
  NULL,
  [3b_umpire_id],
  "U3B"
FROM game_log
WHERE [3b_umpire_id] IS NOT NULL

UNION

SELECT
  game_id,
  NULL,
  lf_umpire_id,
  "ULF"
FROM game_log
WHERE lf_umpire_id IS NOT NULL

UNION

SELECT
  game_id,
  NULL,
  rf_umpire_id,
  "URF"
FROM game_log
WHERE rf_umpire_id IS NOT NULL

UNION

SELECT
  game_id,
  v_name,
  v_manager_id,
  "MM"
FROM game_log
WHERE v_manager_id IS NOT NULL

UNION

SELECT
  game_id,
  h_name,
  h_manager_id,
  "MM"
FROM game_log
WHERE h_manager_id IS NOT NULL

UNION

SELECT
  game_id,
  CASE
      WHEN h_score > v_score THEN h_name
      ELSE v_name
      END,
  winning_pitcher_id,
  "AWP"
FROM game_log
WHERE winning_pitcher_id IS NOT NULL

UNION

SELECT
  game_id,
  CASE
      WHEN h_score < v_score THEN h_name
      ELSE v_name
      END,
  losing_pitcher_id,
  "ALP"
FROM game_log
WHERE losing_pitcher_id IS NOT NULL

UNION

SELECT
  game_id,
  CASE
      WHEN h_score > v_score THEN h_name
      ELSE v_name
      END,
  saving_pitcher_id,
  "ASP"
FROM game_log
WHERE saving_pitcher_id IS NOT NULL

UNION

SELECT
  game_id,
  CASE
      WHEN h_score > v_score THEN h_name
      ELSE v_name
      END,
  winning_rbi_batter_id,
  "AWB"
FROM game_log
WHERE winning_rbi_batter_id IS NOT NULL

UNION

SELECT
  game_id,
  v_name,
  v_starting_pitcher_id,
  "PSP"
FROM game_log
WHERE v_starting_pitcher_id IS NOT NULL

UNION

SELECT
  game_id,
  h_name,
  h_starting_pitcher_id,
  "PSP"
FROM game_log
WHERE h_starting_pitcher_id IS NOT NULL;'
  
dbExecute(conn, person_appearance_populate)

For handling the offensive and defensive positions for both teams, a nested for loop will be used to speed up the process.

In [ ]:
for (letter in c("h", "v")) {
  for (num in 1:9) {
    template <- '
      INSERT INTO person_appearance (
          game_id,
          team_id,
          person_id,
          appearance_type_id
      ) 
          SELECT
              game_id,
              %s_name,
              %s_player_%f_id,
              "O%f"
          FROM game_log
          WHERE %s_player_%f_id IS NOT NULL
      
      UNION
      
          SELECT
              game_id,
              %s_name,
              %s_player_%f_id,
              "D" || CAST(%s_player_%f_def_pos AS INT)
          FROM game_log
          WHERE %s_player_%f_id IS NOT NULL;
    '
    # replace all of the %s and %f with the correct letter number
    template <- gsub("%s", letter, template, fixed = TRUE)
    template <- gsub("%f", num, template, fixed = TRUE)
    
    dbExecute(conn, template)
  }
}

With the normalized tables now all created, and our data inserted, we will now remove the original tables we created when we initially imported the CSV files.

In [ ]:
dbListTables(conn)

tables_to_drop <- c("game_log", "park_codes",
            "team_codes", "person_codes")
            
for (each in tables_to_drop) {
  drop_table = sprintf("DROP TABLE %s", each)
  dbExecute(conn, drop_table)
}


dbListTables(conn)

## Ending the project

With the database creation task now successfully completed, we'll end here and disconnect from the database.

In [ ]:
dbDisconnect(conn)